### Yandex 

In [1]:
pwd

'C:\\Users\\gauthieca\\Desktop\\Code_These\\bandits-to-rank\\Test\\KDD'

In [1]:
from __future__ import print_function
import sys 
import time 
from pyclick.click_models.Evaluation import LogLikelihood, Perplexity
from pyclick.click_models.UBM import UBM
from pyclick.click_models.DBN import DBN
from pyclick.click_models.SDBN import SDBN
from pyclick.click_models.DCM import DCM
from pyclick.click_models.CCM import CCM
from pyclick.click_models.CTR import DCTR, RCTR, GCTR
from pyclick.click_models.CM import CM
from pyclick.click_models.PBM import PBM
from pyclick.utils.Utils import Utils
from pyclick.utils.YandexRelPredChallengeParser import YandexRelPredChallengeParser

from pyclick.search_session.SearchResult import SearchResult
from pyclick.click_models.task_centric.TaskCentricSearchSession import TaskCentricSearchSession

from random import sample
from random import random

In [2]:
import pandas as pd

In [3]:
import sys
sys.path.insert(0, 'C:\\Users\\gauthieca\\Desktop\\Code_ Thèse\\bandits-to-rank')

from bandits_to_rank.sampling.pbm_inference import *
import numpy as np

## Import de Yandex 

In [8]:
yandex_train = pd.read_csv('Yandex_filtered_compress.zip')

In [9]:
top_60_URLID_yandex = []
with open("top_60_URLID_yandex.txt", "r") as f:
    for line in f:
        top_60_URLID_yandex.append(int(line.strip()))

In [10]:
top_60_URLID_yandex[:3]

[4102451, 5681275, 4394913]

In [11]:
yandex_train.shape

(6144, 7)

In [12]:
yandex_train.head()

,Unnamed: 0,dict_index,pos,Impression,Click,query,url
0,0,10042473_1015532,6,17698,47,10042473,1015532
1,1,10042473_1015532,5,5802,28,10042473,1015532
2,2,10042473_1015532,7,13656,44,10042473,1015532
3,3,10042473_13862567,2,1,1,10042473,13862567
4,4,10042473_14937742,3,28501,286,10042473,14937742


In [13]:
group = yandex_train.groupby('query')['Impression'].agg(sum)
print(group.shape)
group.head()

(60,)


query
1435646    149889
1856912    237991
2588400    174081
3276646    425258
3446059    299290
Name: Impression, dtype: int64

### Fonction for get matrice

In [14]:
def get_list_clean_url(df_query,pos_min):

    ###get ad over 1000 :
    series_query_group_ad_Pos = df_query.groupby(["query",'url','pos'])['Impression'].agg(sum)
    df_query_group_ad_Pos=series_query_group_ad_Pos.to_frame()
    over_10000 =  df_query_group_ad_Pos['Impression']>=1000
    df_query_over1000 = df_query_group_ad_Pos[over_10000]

    ### get ad on all 3position : 
    df_query_over1000_ad = df_query_over1000.groupby(["query",'url']).count()
    on_3pos =  df_query_over1000_ad['Impression']==pos_min
    df_query_group_ad_3 = df_query_over1000_ad[on_3pos]
    
    ###List Add :
    ads_list = list(df_query_group_ad_3.index.get_level_values('url'))
    return ads_list 

In [16]:
def buil_mat (df_mat,type_mat,eps = 10**-6.):
    nb_ad,nb_pos =df_mat.shape
    mat =np.zeros((nb_ad, nb_pos), dtype=np.float)
    dict_ad_ind={}
    ad_num=0
    for index, row in df_mat.iterrows():  
        for pos in range(nb_pos):
            dict_ad_ind[index]=ad_num
            if type_mat=='Impression': 
                mat[ad_num][pos]=row[pos]+2*eps
            else :
                mat[ad_num][pos]=row[pos]+eps
        ad_num+=1
    return dict_ad_ind,mat

In [23]:
def getmat (query,df_full,minpos=10 ):
    ###filter by query
    df_query= df_full[df_full['query']==query]
   
    ### filter by ad
    #ad_list = get_list_clean_ad(df_query,minpos)
    #df_query_clean = df_query[df_query.AdId.isin(ad_list)]
    
    #### GET DF impression and click
    df_mat_query_impr = df_query[['url','pos','Impression']].groupby(['url','pos'],as_index = False)['Impression'].agg(sum).pivot('url','pos').fillna(0)
    df_mat_query_clic = df_query[['url','pos','Click']].groupby(['url','pos'],as_index = False).sum().pivot('url','pos').fillna(0)
    
    
    #### Transform to np_array
    dict_ad_ind,mat_query_impr = buil_mat(df_mat_query_impr,'Impression')
    dict_ad_ind,mat_query_clic = buil_mat(df_mat_query_clic,'Clic')
    return mat_query_impr,mat_query_clic,dict_ad_ind
    

In [27]:
def get_param_query(query,df_full):
    impr,click,_ = getmat(query,df_full)
    nb_products,nb_positions = impr.shape
    svd =SVD(nb_products, nb_positions)
    svd.nb_views = impr
    svd.nb_clicks =click
    #print (impr)
    #print (click)

    svd.learn()
   
    return  svd.get_params() 

In [25]:
def get_query(df):
    query = list (df.groupby(['query']).count().index.values)
    return query

In [20]:
query = get_query(yandex_train)

In [21]:
query[:5]

[1435646, 1856912, 2588400, 3276646, 3446059]

In [28]:
dict_theta_query ={}
for i in query:
    theta, kappa =get_param_query(i,yandex_train)
    dict_theta_query[i]={'thetas':list(theta),'kappas':list(kappa)}

In [30]:
dict_theta_query.keys()

dict_keys([1435646, 1856912, 2588400, 3276646, 3446059, 3610030, 3812356, 4102451, 4103811, 4394913, 4455201, 4605190, 4605457, 4831705, 5090291, 5181927, 5353237, 5681275, 6000158, 6052488, 6052895, 7435209, 7631095, 8107157, 8117795, 8152378, 8354851, 9776920, 9801058, 9814521, 9819082, 10042473, 10509813, 10751928, 11113019, 11401789, 11483526, 11618338, 11686152, 13838503, 14200002, 14745379, 15244897, 15566416, 15577854, 16133379, 17670982, 17945604, 18281156, 18980863, 19236076, 19252814, 20015381, 20098967, 20100007, 20303941, 20306025, 20649304, 21291148, 21307448])

In [33]:
def ordonner_dico(dico, liste_ordre):
    dico_ordonne = {}
    for i in liste_ordre:
        dico_ordonne[i]=dico[i]
    return dico_ordonne

In [35]:
dict_theta_query_ordonne =ordonner_dico(dict_theta_query,top_60_URLID_yandex)

In [36]:
dict_theta_query_ordonne.keys()

dict_keys([4102451, 5681275, 4394913, 14200002, 15577854, 4605457, 6052895, 20100007, 10509813, 8107157, 21307448, 9814521, 4605190, 3276646, 8117795, 11113019, 11483526, 10751928, 4455201, 6052488, 10042473, 20649304, 5353237, 20015381, 11686152, 21291148, 3446059, 17670982, 15566416, 20306025, 15244897, 4103811, 5181927, 9819082, 14745379, 13838503, 6000158, 1856912, 3812356, 9801058, 18281156, 3610030, 7631095, 19236076, 18980863, 19252814, 7435209, 9776920, 11401789, 1435646, 20098967, 4831705, 20303941, 11618338, 17945604, 5090291, 2588400, 8152378, 8354851, 16133379])

### Record param

In [6]:
pwd

'C:\\Users\\gauthieca\\Desktop\\Code_These\\bandits-to-rank\\Test\\KDD'

In [7]:
cd Test/KDD/

[WinError 3] Le chemin d’accès spécifié est introuvable: 'Test/KDD/'
C:\Users\gauthieca\Desktop\Code_These\bandits-to-rank\Test\KDD


In [40]:
import json
with open('param_Yandex.txt', 'w') as file:
    json.dump(dict_theta_query_ordonne, file)

### Yandex CM

In [8]:
click_model = CM()

In [9]:
cd ../../..

C:\Users\gauthieca\Desktop\Code_These


In [133]:
search_sessions = YandexRelPredChallengeParser().parse('./PyClick-master/PyClick-master/examples/data/Yandex_short.csv', 10000000)

In [134]:
search_sessions[1]

{"query": "0", "task": "70", "web_results": [{"click": 0, "id": "2106856"}, {"click": 0, "id": "34175267,3279130"}, {"click": 0, "id": "34033472,3277148"}, {"click": 0, "id": "31455963,3084739"}, {"click": 0, "id": "43630531,3823198"}, {"click": 0, "id": "39008410,3549024"}, {"click": 0, "id": "52754664,4306706"}, {"click": 0, "id": "11352247,1125990"}, {"click": 0, "id": "19715603,2026752"}, {"click": 0, "id": "1045409,156610"}, {"click": 0, "id": "60010091,4668800"}, {"click": 0, "id": "34175267,3279130,34033472,3277148,31455963,3084739,43630531,3823198,39008410,3549024,52754664,4306706,11352247,1125990_19715603,2026752_1045409,156610_60010091,4668800"}]}

In [136]:
train_test_split = int(len(search_sessions) * 0.99)
train_sessions = search_sessions[:train_test_split]
train_queries = Utils.get_unique_queries(train_sessions)

test_sessions = Utils.filter_sessions(search_sessions[train_test_split:], train_queries)
test_queries = Utils.get_unique_queries(test_sessions)


In [137]:
start = time.time()
param = click_model.train(train_sessions)
end = time.time()
print ("\tTrained %s click model in %i secs:\n%r" % (click_model.__class__.__name__, end - start, click_model))

	Trained CM click model in 109 secs:
CMParamNames.attr
2: {'2223608,4291604': 0.0001, '39061378,3551649': 0.0001, '29220341,2863227': 0.0001, '2522212,362401': 0.0001, '27398472,2655040': 0.0001, '36612684,3409847': 0.0001, '21250468,2179127': 0.0001, '2522317,362435': 0.0001, '27221002,2637612': 0.0001, '69693878,5153755': 0.0001, '1951834': 0.0000, '34175267,3279130': 0.0000, '34033472,3277148': 0.0000, '11352247,1125990': 0.0000, '43630531,3823198': 0.0000, '32363511,3142702': 0.0000, '26202930,2597528': 0.0000, '67804805,5088102': 0.1667, '7920148,943850': 0.0000, '32694554,3182011': 0.0000, '1045409,156610': 0.0000, '60010091,4668800': 0.0000, '16286311,1697410': 0.0002, '3991477,2452511,4143827': 0.0001, '29464699,2863926': 0.0001, '29408102,2863781': 0.0001, '29516474,2864015': 0.0001, '29328434,2863521': 0.0001, '29530804,2864085': 0.0001, '29302552,2863473': 0.0001, '29451601,2863817': 0.0001, '63371986,4843230': 0.0001, '29276408,2863419': 0.0001, '4601785,4098689': 0.0001, '

In [138]:
type(click_model.params)

dict

In [139]:
param_Yandex_CM =click_model.params

In [140]:
for i in param_Yandex_CM.keys():
    print(type(param_Yandex_CM[i]._container))
    dico_param = param_Yandex_CM[i]._container
    param_Yandex_CM[i]

<class 'collections.defaultdict'>


In [141]:
dico_param.keys()

dict_keys(['2', '1', '5', '3', '4', '6', '9', '8', '10', '11', '12', '7', '17', '16', '18', '19', '14', '13', '39', '41', '25', '26', '32', '33', '34', '23', '24', '15', '21', '27', '29', '20', '28', '22', '35', '36', '37', '48', '50', '46', '51', '59', '63', '40', '30', '31', '38', 1, '0'])

In [143]:
del dico_param['0']

In [148]:
dico_param_int ={}
for key, value in dico_param.items():
    #print("key",key)
    #print("###############")
    dico_param_int[key]={}
    for key_param, value_param in value.items():
       # print(key_param)
        str_value =str(value_param)
       #print(key_param,str_value)
        dico_param_int[key][key_param] = float(str_value)
        #print(dico_param_int[key][key_param])

In [149]:
dico_param['2']['70987384,5273765']

0.0011

In [150]:
dico_param_order ={}
for key, value in dico_param_int.items():
     dico_param_order[key]=sorted(value.items(), key=lambda t: t[1])[::-1]

In [201]:
dico_param_liste = {}
list_exist = []
for key_query, value in dico_param_int.items():
    liste_theta_query_full = []
    for _,value_param in value.items():
        liste_theta_query_full.append(value_param)
    liste_theta_query_unique = list(set(liste_theta_query_full))
    liste_theta_query_unique.sort()
    top_20 = liste_theta_query_unique[::-1][:20]
    if (top_20 not in list_exist) and len(top_20)>10:
        dico_param_liste[key_query] = top_20
        list_exist.append(top_20)

In [202]:
dico_param_liste

{'2': [0.5,
  0.3333,
  0.2,
  0.1667,
  0.1429,
  0.1111,
  0.1,
  0.0909,
  0.0769,
  0.0714,
  0.0667,
  0.0625,
  0.0588,
  0.0556,
  0.0526,
  0.0476,
  0.0455,
  0.0435,
  0.0417,
  0.04],
 '1': [0.5,
  0.3333,
  0.2,
  0.1667,
  0.1429,
  0.1111,
  0.1,
  0.0909,
  0.0769,
  0.0714,
  0.0667,
  0.0588,
  0.0556,
  0.0526,
  0.0476,
  0.0455,
  0.0435,
  0.04,
  0.0385,
  0.037],
 '5': [0.5,
  0.3333,
  0.2,
  0.1667,
  0.1429,
  0.1111,
  0.1,
  0.0909,
  0.0769,
  0.0714,
  0.0667,
  0.0588,
  0.0556,
  0.0526,
  0.0476,
  0.0455,
  0.0435,
  0.04,
  0.0385,
  0.0345],
 '6': [0.5,
  0.3333,
  0.2,
  0.1667,
  0.1429,
  0.1111,
  0.1,
  0.0909,
  0.0769,
  0.0714,
  0.0667,
  0.0625,
  0.0588,
  0.0556,
  0.0526,
  0.0476,
  0.0455,
  0.0435,
  0.04,
  0.0385],
 '9': [0.5,
  0.3333,
  0.2,
  0.1667,
  0.1429,
  0.1,
  0.0909,
  0.0714,
  0.0667,
  0.0556,
  0.0455,
  0.0385,
  0.037,
  0.0333,
  0.0294,
  0.0263,
  0.0238,
  0.0217,
  0.02,
  0.0189],
 '8': [0.5,
  0.3333,
  0.2